In [5]:
from datetime import datetime
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from tqdm import tqdm

LEAUGE_TYPE = "NBA"

from OddsAPI import API
api = API()
api.setAPI("O5b2PclgVIZtFEYI0SceTEKU1MUc9lx5XjkqDRXkwU0")

In [6]:
## pivot: converts a dataframe that contains two seperate rows for each player (one for over, one for under) to have one row for each player
def pivot(df):
    df = df.copy()
    pivot_df = pd.pivot_table(df, values='odds', index=['handicap', 'participant_name'], columns='name').reset_index()
    try:
        pivot_df.columns = ['line', 'participant_name', 'over_odds', 'under_odds']
    except Exception as e:
        print(e)
        print(df)
        print(pivot_df)
        print(pivot_df.columns)
    return pivot_df

## calculate odds
## adapted from Ammar Sulmanjee
def calculate_odds(x, y=None):
    if not y:
        if x>= 0: return 1 + x/100
        return 1 + 100/abs(x)

    imp_prob1 = (1 / calculate_odds(x)) * 100
    imp_prob2 = (1 / calculate_odds(y)) * 100

    total_implied_prob = round(imp_prob1 + imp_prob2, 4)
    fair_prob1 = round(imp_prob1 / total_implied_prob * 100, 2)
    fair_prob2 = round(imp_prob2 / total_implied_prob * 100, 2)

    index = "over" if fair_prob1 > fair_prob2 else "under"

    return {"type": index, "ev": max(fair_prob1, fair_prob2) - 50}

def download_bets(best_bets):
    best_bets['market'] = best_bets['market'].str.replace(r'_|player|over_under', '', regex=True).str.strip()
    best_bets = best_bets.drop(columns=['game'])
    best_bets.to_csv(f"Bets_{datetime.now().strftime('%Y-%m-%d')}.csv", index=None)

def check_player(raw_bet_data, name, market, line=None):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)  
    if line:
        display(raw_bet_data[(raw_bet_data['participant_name'] == name) & (raw_bet_data['market'] == market) & (raw_bet_data['handicap'] == line)])
        pd.reset_option('display.max_rows')
        pd.reset_option('display.max_columns')
        return
    display(raw_bet_data[(raw_bet_data['participant_name'] == name) & (raw_bet_data['market'] == market)])
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')

In [7]:
### Pull Underdog Lines
# result: ud (DataFrame) contains underdog lines + names, games_to_check (dictionary) contains each game, with its corresponding markets to check

ud = pd.DataFrame.from_dict(api.get_fantasy_lines(LEAUGE_TYPE)['fantasy_books'])

ud = ud[ud['bookie_key'] == "underdog"]
ud = pd.DataFrame.from_dict(list(ud["markets"])[0])
ud.columns = ['markets', 'lines']

df = pd.DataFrame(columns=['participant_name', 'line', 'game_id', 'market'])

lines = list(ud['lines'])
markets = list(ud['markets'])

for x, line in enumerate(lines):
    for prop in line:
        temp = pd.DataFrame()

        temp['participant_name'] = [prop['participant_name']]
        temp['line'] = [prop['line']]
        temp['game_id'] = [prop['game_id']]
        temp['market'] = markets[x]

        df = pd.concat([df, temp], axis=0)

df = df.reset_index()
df = df.drop(columns=['index'])
df = df.sort_values(by=['game_id'])


ud = df[['participant_name', 'line', 'market']]
ud.columns = [ud.columns[0], 'handicap', 'market']

result_df = df.groupby('game_id')['market'].agg(list).reset_index()
games_to_check = dict(zip(result_df['game_id'], result_df['market']))

In [8]:
bets = pd.DataFrame(columns = ['participant_name', 'ev', 'market', 'line'])
raw_bet_data = pd.DataFrame()

In [10]:
for game, markets in tqdm(games_to_check.items(), desc="Processing games"):
    for market in markets:
            
        # get sportsbook odds
        try:
            bookies_data = api.get_most_recent_odds(game, market)
        except Exception as e:
            if str(e) == "ERROR 422": continue
            else: 
                print(e)
                continue
        books = []
        
        for i in range(len(bookies_data['sportsbooks'])):
            df = pd.DataFrame.from_dict(bookies_data['sportsbooks'][i]['market']['outcomes'])
            df = df[df['timestamp'].str.contains(datetime.now().strftime('%Y-%m-%d'))]

            df['book'] = bookies_data['sportsbooks'][i]['bookie_key']
            df['market'] = market

            books.append(df)

        raw_data = pd.concat(books)
        raw_bet_data = pd.concat([raw_bet_data, raw_data])

        # concat the books into one
        new_datasets = []
        for dataset in books:
            dataset = pd.merge(dataset[['handicap', 'odds', 'participant_name', 'name', 'market']], ud, how='inner', on = ['participant_name', 'handicap' , 'market'])
            dataset = dataset[['handicap', 'odds', 'participant_name', 'name']]
            
            if dataset.shape[0] > 0 and dataset.shape[1] > 0 and dataset.shape[0]%2 == 0:
                dataset = pivot(dataset)

                dataset = dataset[abs(dataset['over_odds']) < 250]
                dataset = dataset[abs(dataset['under_odds']) < 250]
                
                if dataset.shape[0] > 0 and dataset.shape[1] > 0:
                    new_datasets.append(dataset)
        if new_datasets == []: continue

        
        final = pd.concat(new_datasets, axis=0)
        final = final.dropna()

        final[['type', 'ev']] = final.apply(lambda row: pd.Series(calculate_odds(row['over_odds'], row['under_odds'])), axis=1)
        final = final.groupby(['participant_name', 'line', 'type']).agg({'ev': 'median', 'over_odds':'median', 'under_odds':'median'}).reset_index()

        final['market'] = market
        final['game'] = game


        if final.shape[0] > 0: 
            bets = pd.concat([bets, final[['participant_name', 'ev', 'type', 'market', 'line', 'over_odds', 'under_odds', 'game']]])

Processing games:  17%|█▋        | 1/6 [00:49<04:08, 49.67s/it]

In [75]:
bets = bets.sort_values(by=['ev'], ascending=False)
bets = bets.dropna()
bets = bets.drop_duplicates(keep='first', ignore_index=True)

best_bets = bets.copy()
best_bets = best_bets[best_bets['ev'] > 4.5]

In [76]:
print(f"Total Bets found!: {bets.shape[0]}")
print(f"Total Positive EV Bets found!: {best_bets.shape[0]}")

Total Bets found!: 635
Total Positive EV Bets found!: 44


In [77]:
display(best_bets.head())
display(best_bets.tail())

,participant_name,ev,market,line,type,over_odds,under_odds,game
0,Lauri Markkanen,15.215,player_turnovers_over_under,1.5,under,170.0,-227.5,6a84df3782bc046cf5bda47fc177862d
1,Jordan Clarkson,14.855,player_blocks_steals_over_under,0.5,over,-222.5,167.5,6a84df3782bc046cf5bda47fc177862d
2,Keyonte George,14.855,player_steals_over_under,0.5,under,167.5,-225.0,fb5faee746c6258a3c864d78ff1f37f2
3,Keyonte George,14.365,player_assists_over_under,3.5,under,160.0,-227.5,fb5faee746c6258a3c864d78ff1f37f2
4,Lauri Markkanen,13.34,player_blocks_over_under,0.5,under,155.0,-210.0,fb5faee746c6258a3c864d78ff1f37f2


,participant_name,ev,market,line,type,over_odds,under_odds,game
39,Pascal Siakam,4.845,player_assists_points_rebounds_over_under,29.5,over,-142.0,107.0,cc85b55674e0d8863186b26f84145b71
40,Keyonte George,4.8,player_assists_points_over_under,12.5,under,107.0,-141.5,fb5faee746c6258a3c864d78ff1f37f2
41,Klay Thompson,4.77,player_threes_over_under,3.5,over,-140.0,107.0,7aab939af74de8c8d7726c83a47ed6a3
42,Pascal Siakam,4.65,player_assists_points_over_under,23.5,over,-142.0,106.0,cc85b55674e0d8863186b26f84145b71
43,Klay Thompson,4.58,player_threes_over_under,3.5,over,-140.0,106.0,7aab939af74de8c8d7726c83a47ed6a3


In [84]:
download_bets(best_bets)

In [7]:
check_player(raw_bet_data, 'Lauri Markkanen', 'player_turnovers_over_under', 1.5)

,timestamp,handicap,odds,participant,participant_name,name,description,book,market
9,2024-01-17T16:05:02,1.5,175,16315.0,Lauri Markkanen,Over,Over - Lauri Markkanen Turnovers,draftkings,player_turnovers_over_under
22,2024-01-17T16:05:02,1.5,-230,16315.0,Lauri Markkanen,Under,Under - Lauri Markkanen Turnovers,draftkings,player_turnovers_over_under
10,2024-01-17T16:20:02,1.5,175,16315.0,Lauri Markkanen,Over,Over 1.5 Lauri Markkanen (UTA): Turnovers,betmgm,player_turnovers_over_under
23,2024-01-17T16:05:04,1.5,-250,16315.0,Lauri Markkanen,Under,Under 1.5 Lauri Markkanen (UTA): Turnovers,betmgm,player_turnovers_over_under
7,2024-01-17T17:00:02,1.5,165,16315.0,Lauri Markkanen,Over,Lauri Markkanen Over 1.5 Lauri Markkanen Over ...,pointsbet,player_turnovers_over_under
18,2024-01-17T17:00:02,1.5,-225,16315.0,Lauri Markkanen,Under,Lauri Markkanen Under 1.5 Lauri Markkanen Unde...,pointsbet,player_turnovers_over_under
9,2024-01-17T16:05:02,1.5,175,16315.0,Lauri Markkanen,Over,Over - Lauri Markkanen Turnovers,draftkings,player_turnovers_over_under
22,2024-01-17T16:05:02,1.5,-230,16315.0,Lauri Markkanen,Under,Under - Lauri Markkanen Turnovers,draftkings,player_turnovers_over_under
10,2024-01-17T16:20:02,1.5,175,16315.0,Lauri Markkanen,Over,Over 1.5 Lauri Markkanen (UTA): Turnovers,betmgm,player_turnovers_over_under
23,2024-01-17T16:05:04,1.5,-250,16315.0,Lauri Markkanen,Under,Under 1.5 Lauri Markkanen (UTA): Turnovers,betmgm,player_turnovers_over_under


In [1]:
a = raw_bet_data[(raw_bet_data['participant_name'] == 'Lauri Markkanen') & (raw_bet_data['market'] == 'player_turnovers_over_under') & (raw_bet_data['handicap'] == 1.5)].copy()
a = a[a['timestamp'].contains(datetime.now().strftime('%Y-%m-%d'))] 

NameError: name 'raw_bet_data' is not defined

'2024-01-30'